In [30]:

import os

import socket

import numpy as np 
import pandas as pd

a_file = './A/A.csv'
b_file = './B/B.csv'
c_file = './F/F.csv'

T0 = 5*60

In [31]:

def read_data(file, my_socket = None):

    my_socket = socket.gethostbyname(socket.gethostname()) if my_socket is None else my_socket

    data = pd.read_csv(file, sep=';')

    data = data[data['Sour IP'] != socket]

    #data = data[data['Dest IP'] == socket] 

    return data


A = read_data('./A/A.csv')

A.head()

,Sour IP,Dest IP,Numb P,Data,Sour Port,Dest Port,Time
0,192.133.154.32,NaN,12,263,13704,11469,0:57:36
1,192.111.168.170,NaN,2,138,43414,41879,0:43:12
2,192.145.203.65,NaN,14,252,22513,54661,1:40:48
3,192.83.225.215,NaN,3,135,8158,11146,1:40:48
4,192.98.242.158,NaN,9,89,31788,48719,0:57:36


In [32]:
def time2number(time: str):
    h, m, s = [int(t) for t in time.split(':')]
    return s + 60*m + 3600*h

In [33]:

def AtoB(a_file = './A/A.csv', b_file = './B/B.csv', my_socket = None):

    A = read_data(a_file, my_socket)
    if os.path.exists(b_file):
        B = read_data(b_file, my_socket)
        A = pd.concat([A, B], ignore_index=True)

    # можно и из А убрать те 5 минут
    print(A.columns)
    times = np.array([time2number(time) for time in A['Time '].values])
    time_mask = times > (times.max() - 5*60)

    A.loc[time_mask,:].to_csv(b_file, index=False,  sep=';')

    return A
    

F = AtoB()

F.head()

Index(['Sour IP', 'Dest IP', 'Numb P', 'Data', 'Sour Port', 'Dest Port',
       'Time '],
      dtype='object')


,Sour IP,Dest IP,Numb P,Data,Sour Port,Dest Port,Time
0,192.133.154.32,NaN,12,263,13704,11469,0:57:36
1,192.111.168.170,NaN,2,138,43414,41879,0:43:12
2,192.145.203.65,NaN,14,252,22513,54661,1:40:48
3,192.83.225.215,NaN,3,135,8158,11146,1:40:48
4,192.98.242.158,NaN,9,89,31788,48719,0:57:36


In [34]:

mask_Numb = F['Numb P'] < 5
if np.logical_not(mask_Numb).sum() == F.shape[0]:
    F.to_csv(c_file, sep=';', index=False)
    os.remove(a_file)
else:
    F = F[mask_Numb]

F.head()

,Sour IP,Dest IP,Numb P,Data,Sour Port,Dest Port,Time
1,192.111.168.170,NaN,2,138,43414,41879,0:43:12
3,192.83.225.215,NaN,3,135,8158,11146,1:40:48
5,192.41.29.75,NaN,2,79,60845,23940,1:40:48
13,192.11.165.113,NaN,3,131,24867,45029,1:55:12
18,192.209.96.137,NaN,3,199,2452,16027,2:24:00


In [35]:
F.shape

(101, 7)

In [36]:
F['time'] = [time2number(time) for time in F['Time ']]

F.sort_values(by = ['Sour IP', 'time'], inplace = True)

F.head()

,Sour IP,Dest IP,Numb P,Data,Sour Port,Dest Port,Time,time
276,192.100.15.85,NaN,3,244,53751,12529,0:43:12,2592
23,192.105.223.33,NaN,1,234,25508,1314,0:43:12,2592
189,192.106.138.125,NaN,4,268,30736,31660,1:40:48,6048
13,192.11.165.113,NaN,3,131,24867,45029,1:55:12,6912
1,192.111.168.170,NaN,2,138,43414,41879,0:43:12,2592


In [37]:


Ds = [F[F['Sour IP'] == sour] for sour in pd.unique(F['Sour IP'])]


def T_setter(data):

    T = np.empty(data.shape[0])
    for i in range(1, data.shape[0]):
        T[i] = data['time'][i] - data['time'][i-1]

    T[0] = np.mean(T[1:])

    data['T2'] = T0 - T
    return data


#F = pd.concat([T_setter(df) for df in Ds])

toSee = [T_setter(df) for df in Ds]

C:\Users\qtckp\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\qtckp\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-37-327305e17b69>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['T2'] = T0 - T


KeyError: 1